In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    pred_threshold: 0.5 

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0 
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-area_estimation_error'
    # checkpoint: 'min-val-ece_loss'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ISLES_3D_Eval"

base_model: 
    - "/storage/vbutoi/scratch/ESE/training/09_09_24_ISLES_3D_Dice"

inference_data:
    _class: "ese.datasets.ISLES"
    version: 1.0
    split: ("cal", "val")

dataloader:
    batch_size: 1

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(inf_cfgs)

4

## Running Jobs

In [8]:
from ese.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(meth

In [9]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[-1], 
    job_func=get_cal_stats,
    run_name='debug',
    # show_examples=True,
    show_examples=True,
    gpu='0',
    # gpu='4',
) 

/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 40
Loading checkpoint: min-val-area_estimation_error.
[Errno 2] No such file or directory: '/storage/vbutoi/scratch/ESE/training/09_09_24_ISLES_3D_Dice/20240909_144644-PN8W-372f4c4cd64eddba01c0bc3f70f15c24/checkpoints/min-val-area_estimation_error.pt'
Defaulting to loading: max-val-dice_score.
Running:

calibrator:
  _name: None
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  exp_name: ISLES_3D_Eval
  exp_root: /storage/vbutoi/scratch/ESE/inference/09_12_24_ISLES_3D_Eval
  inf_dataset_name: ISLES
  inference_seed: 40
  model_dir: /storage/vbutoi/scratch/ESE/training/09_09_24_ISLES_3D_Dice/20240909_144644-PN8W-372f4c4cd64eddba01c0bc3f70f15c24
  pred_threshold: 0.5
  pretrained_seed: 41
global_cal_metrics:
- ECE:
    _fn: ese.metrics.calibration.ece_loss
    cal_type: toplabel
global_calibration:
  neighborhood_width: 3
  num_classes: 1
  num_prob_bins: 15
image_cal_metrics:
- Image_ECE:
    _fn: ese.metrics.calibration.image_ece_loss
    cal_type:

/storage/vbutoi/projects/ionpy/experiment/train.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=self.device)


torch.Size([1, 1, 148, 180, 71]).00%)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )